In [10]:
# html 코드 그대로 크롤링 할시 주석도 같이 추출됨

from bs4 import BeautifulSoup
import sys
import requests
import re
import pandas as pd
import csv
import os
import openai

openai.api_key ="API_KEY"

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

def extract_tables(html_str):
    # 정규표현식 패턴 정의
    pattern = r'(<table.*?>.*?</table>)'
    # 정규표현식으로 <table>...</table> 부분 찾기
    tables = re.findall(pattern, html_str, re.DOTALL)  # re.DOTALL은 줄바꿈 문자도 포함해서 찾도록 합니다.
    return tables

def table_function(table_g):
    sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>대영역</th>
        <th>세부영역</th>
        <th colspan="2">학점</th>
    </tr>
    <tr>
        <td>필수교양</td>
        <td>
            국어<br>
            영어<br>
            소프트웨어<br>
            항공우주특성화
        </td>
        <td colspan="2">
            공과대학 13 학점<br>
            항경대학 12 학점
        </td>
    </tr>
    <tr>
        <td>전공기초교양</td>
        <td>전공기초</td>
        <td colspan="2">0 ~ 18학점, 각 학부(과)별 상이</td>
    </tr>
    <tr>
        <td>핵심교양</td>
        <td>
            1) 철학과 역사<br>
            2) 문학과 예술<br>
            3) 인간과 사회<br>
            4) 자연과 과학<br>
            5) 실용 및 글로벌
        </td>
        <td>9 학점 이상<br>학부(과)별 지정영역이수<br>(4-1 참조)</td>
        <td rowspan="2">12 학점<br>※ 1) ~ 5) 학부(과)별 지정 영역 중<br>3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수<br>※ 세부영역 중복, 대영역 미중복시 인정됨</td>
    </tr>
    <tr>
        <td>심화교양</td>
        <td>
            6) 철학과 역사<br>
            7) 문학과 예술<br>
            8) 인간과 사회<br>
            9) 자연과 과학<br>
            10) 실용 및 글로벌<br>
            11) 봉사와 리더십<br>
            12) 자기계발<br>
            13) 외국어(영어)
        </td>
        <td>자유이수</td>
    </tr>
    <tr>
        <th colspan="2">계</th>
        <th colspan="2">24 ~ 43 학점</th>
    </tr>
</tbody></table>
---------
답 : 
---------
대영역 : 필수교양
세부영역 : 국어, 영어, 소프트웨어, 항공우주특성화
학점 : 공과대학 13 학점, 항경대학 12 학점

대영역 : 전공기초교양
세부영역 : 전공기초
학점 : 0 ~ 18학점, 각 학부(과)별 상이

대영역 : 핵심교양
세부영역 : 
1) 철학과 역사
2) 문학과 예술
3) 인간과 사회
4) 자연과 과학
5) 실용 및 글로벌
학점 : 9 학점 이상, 학부(과)별 지정영역이수 (4-1 참조)
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

대영역 : 심화교양
세부영역 : 
6) 철학과 역사
7) 문학과 예술
8) 인간과 사회
9) 자연과 과학
10) 실용 및 글로벌
11) 봉사와 리더십
12) 자기계발
13) 외국어(영어)
학점 : 자유이수
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

계 : 24 ~ 43 학점
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k-0613",
      messages=[
      {"role": "system", "content": f'"{sys}"'},
    ], temperature =  0.001)
    answer = completion['choices'][0]['message']['content']
    if answer.endswith('"'):
        answer = answer[:-1]
        answer = answer
    return answer 

# 홈페이지 주소 가져오기
# url = "https://kau.ac.kr/web/pages/gc15707h.do"
url = """https://kau.ac.kr/web/pages/gc95047h.do
https://kau.ac.kr/web/pages/gc81492h.do
https://kau.ac.kr/web/pages/gc45400h.do
https://kau.ac.kr/web/pages/gc51173h.do
https://kau.ac.kr/web/pages/gc74039h.do
https://kau.ac.kr/web/pages/gc6309h.do
https://kau.ac.kr/web/pages/gc61422h.do
https://kau.ac.kr/web/pages/gc84448h.do
https://kau.ac.kr/web/pages/gc15707h.do
https://kau.ac.kr/web/pages/gc18564h.do
https://kau.ac.kr/web/pages/gc21891h.do
https://kau.ac.kr/web/pages/gc86171h.do
https://kau.ac.kr/web/pages/gc1021h.do
https://kau.ac.kr/web/pages/gc78248h.do
https://kau.ac.kr/web/pages/gc63682h.do
https://kau.ac.kr/web/pages/gc86720h.do
https://kau.ac.kr/web/pages/gc40674h.do
https://kau.ac.kr/web/pages/gc29385h.do
https://kau.ac.kr/web/pages/gc34817h.do
https://kau.ac.kr/web/pages/gc3720h.do
https://kau.ac.kr/web/pages/gc1463h.do
https://kau.ac.kr/web/pages/gc62866h.do
https://kau.ac.kr/web/pages/gc25111h.do
https://kau.ac.kr/web/pages/gc29040h.do
https://kau.ac.kr/web/pages/gc20177h.do
https://kau.ac.kr/web/pages/gc12217h.do
https://kau.ac.kr/web/pages/gc98903h.do
https://kau.ac.kr/web/pages/gc32582h.do
https://kau.ac.kr/web/pages/gc17412h.do
https://kau.ac.kr/web/pages/gc82522h.do
https://kau.ac.kr/web/pages/gc51160h.do
https://kau.ac.kr/web/pages/gc59030h.do
https://kau.ac.kr/web/pages/gc221h.do
https://kau.ac.kr/web/pages/gc88949h.do
https://kau.ac.kr/web/pages/gc98930h.do
https://kau.ac.kr/web/pages/gc62234h.do
https://kau.ac.kr/web/pages/gc37548h.do
https://kau.ac.kr/web/pages/gc3480h.do
https://kau.ac.kr/web/pages/gc66045h.do
https://kau.ac.kr/web/pages/gc52782h.do
https://kau.ac.kr/web/pages/gc66085h.do
https://kau.ac.kr/web/pages/gc53123h.do
https://kau.ac.kr/web/pages/gc8314h.do
https://kau.ac.kr/web/pages/gc60434h.do
https://kau.ac.kr/web/pages/gc30058h.do
https://kau.ac.kr/web/pages/gc36597h.do
"""
url_split = split_sentences(url)
url_list = []

for i in range(len(url_split)):
    url_list.append(url_split[i].strip())
print(url_list)

for jj in range(len(url_list)):
    temp_url = url_list[jj]
    html = requests.get(temp_url)
    bs_html = BeautifulSoup(html.content, "html.parser")

    # print(bs_html) # 웹 페이지 출력
    bs_html = str(bs_html)
    split_text = split_sentences(bs_html)
    tp = 0
    tp_noise = 0
    tp_annotation = 0
    temp_list = []
    for i in range(len(split_text)):
        temp_text = split_text[i]
        if "<html lang" in temp_text and tp_noise == 0:
            tp_noise = 1
        elif "<html lang" in temp_text and tp_noise == 1:
            tp =1
            temp_list.append(temp_text)
#             print(temp_text)
        elif tp == 1 and "</html>" in temp_text:
            break
        elif tp == 1 and "<!--" in temp_text and "-->" in temp_text:
            temp_text = re.sub(r'<!--.*?-->', '', temp_text)
            temp_list.append(temp_text)            
        elif tp == 1 and "<!--" in temp_text and "-->" not in temp_text:
            tp_annotation = 1
            continue
        elif tp == 1 and tp_annotation == 1 and "-->" not in temp_text:
            continue
        elif tp == 1 and tp_annotation == 1 and "-->" in temp_text:
            tp_annotation = 0
            continue
        elif tp == 1:
            temp_list.append(temp_text)
#             print(temp_text)

    print("%%%%%%%%%%%%%%%%%%%")
    
    # table 태그는 하나의 리스트 원소로 두기
    
    table_tp = 0
    table_temp_str = ""
    re_table_list = []
    for i in range(len(temp_list)):
        if "<table" in temp_list[i] and  "</table>" in temp_list[i] and table_tp == 0:  # 한 줄 table 일 때
            test_str = temp_list[i]
            tables = extract_tables(test_str)
            for table in tables:
                # 해당 table이 test_str에서 시작하는 위치 찾기
                start_index = test_str.find(table)
                # 해당 table의 내용을 출력하기 전까지의 test_str의 내용을 출력
                re_table_list.append(test_str[:start_index].strip())
                # 해당 table 출력
                re_table_list.append(table.strip())
                # 이미 출력된 부분을 test_str에서 제거
                test_str = test_str[start_index + len(table):]
                # 남은 test_str에서 내용 출력
            re_table_list.append(test_str.strip())
        elif "<table" in temp_list[i] and table_tp == 0:
            table_tp = 1
            table_temp_str = table_temp_str + temp_list[i] + "\n"            
        elif "</table>" not in temp_list[i] and table_tp == 1:
            table_temp_str = table_temp_str + temp_list[i] + "\n"
        elif "</table>" in temp_list[i] and table_tp == 1:
            table_tp = 0
            table_temp_str = table_temp_str + temp_list[i] + "\n"
            re_table_list.append(table_temp_str.strip())
            table_temp_str = ""
        else:
            re_table_list.append(temp_list[i])
    
    print("-------jj-------")
    print(len(re_table_list))
    print(re_table_list)
# #     # table 태그 임의로 변형 (미완성)
    
#     re_table_list2 = []
#     table_temp_str = ""
#     tr_tp = 0
#     for i in range(len(re_table_list)):
#         if "<table" in re_table_list[i]:
#             table_tag_context = split_sentences(re_table_list[i])
#             for j in range(len(table_tag_context):
#                 if "<tr>" not in table_tag_context[i]
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line + "\n"
#                 elif "<tr>" in table_tag_context[i]
#                     tr_tp = 1
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line
#         else:
#             re_table_list2.append(re_table_list[i])

#  table 태그는 gpt에게 정리
    re_table_list2 = []               
    for i in range(len(re_table_list)):
        if "<table" in re_table_list[i]:
            table_g = re_table_list[i]
            err_count = 0
            try :
                answer = table_function(table_g)
            except Exception as e:
                err_count += 1
                print(err_count)
                if err_count >= 5:
                    print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
                else:
                    print("예외가 발생했습니다. (%s)" % e)
                    answer = table_function(table_g)        
            re_table_list2.append(answer.strip())
        else:
            re_table_list2.append(re_table_list[i])
            
    previous_line = "#$%"
    temp_str = ""
    for i in range(len(re_table_list2)):
        temp_line = re.sub('(<([^>]+)>)', '', re_table_list2[i])
        temp_line = temp_line.strip()
        if previous_line == "" and temp_line == "":
            continue
        else:
            print(temp_line)
            temp_str = temp_str + temp_line + "\n"
        previous_line = temp_line
    temp_str = temp_str.strip()
    f = open(f'url_data.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(jj), temp_str, url_list[jj]])
    f.close()

['https://kau.ac.kr/web/pages/gc95047h.do', 'https://kau.ac.kr/web/pages/gc81492h.do', 'https://kau.ac.kr/web/pages/gc45400h.do', 'https://kau.ac.kr/web/pages/gc51173h.do', 'https://kau.ac.kr/web/pages/gc74039h.do', 'https://kau.ac.kr/web/pages/gc6309h.do', 'https://kau.ac.kr/web/pages/gc61422h.do', 'https://kau.ac.kr/web/pages/gc84448h.do', 'https://kau.ac.kr/web/pages/gc15707h.do', 'https://kau.ac.kr/web/pages/gc18564h.do', 'https://kau.ac.kr/web/pages/gc21891h.do', 'https://kau.ac.kr/web/pages/gc86171h.do', 'https://kau.ac.kr/web/pages/gc1021h.do', 'https://kau.ac.kr/web/pages/gc78248h.do', 'https://kau.ac.kr/web/pages/gc63682h.do', 'https://kau.ac.kr/web/pages/gc86720h.do', 'https://kau.ac.kr/web/pages/gc40674h.do', 'https://kau.ac.kr/web/pages/gc29385h.do', 'https://kau.ac.kr/web/pages/gc34817h.do', 'https://kau.ac.kr/web/pages/gc3720h.do', 'https://kau.ac.kr/web/pages/gc1463h.do', 'https://kau.ac.kr/web/pages/gc62866h.do', 'https://kau.ac.kr/web/pages/gc25111h.do', 'https://kau.a


전과 신청 안내
1. 신청 대상자
구분 : 2학년 전과 - 1학년(33학점이상 이수) 수료 & 2학기 이수자
비고 : [경영학부] - 평점평균3.0 / 토익700점 이상 신청가능
[항공전자정보공학부] - 평점평균3.0 이상 신청가능
[항공운항학과] - 평점평균3.5/토익800점 이상 신청가능

구분 : 3학년 전과 - 2학년(65학점 이상 이수) 수료 & 4학기 이수자
비고 : [경영학부] - 평점평균3.0 / 토익700점 이상 신청가능
[항공전자정보공학부] - 평점평균3.0 이상 신청가능
[항공운항학과] - 평점평균3.5/토익800점 이상 신청가능
휴학생은 전과 신청 전 복학신청을 해야 함
2. 신청기간

매년 소정 기간 내 (1월 중 신청)
자세한 일정은 학사공지 ‘전과 신청안내’ 참고

3. 제출서류 및 제출처
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실

4. 신청절차

전과신청서 제출(각 학부(과) 사무실) → 학부(과) 전과 심의 → 심사결과개별통보
※ 면접 실시 학과는 학과에서 일정 개별 통보

5. 신청서 양식

전과 신청원 양식

문의 : 각 학부(과)   관리 : 교무팀전화번호02-300-0457E-mailkyomu@kau.ac.kr

%%%%%%%%%%%%%%%%%%%
-------jj-------
98
['<html lang="ko">', '<head>', '</head>', '<body>', '<div class="article_box_nrm">', '<h4>학·석사 연계과정 안내</h4>', '<h5>1. 학·석사 연계과정이란?</h5>', '<ul class="nrm_list_n">', '<li>', '학·석사 연계과정은 학사과정과 석사과정을 연계하여 수업연한


제 1 학기
구분 : '23.1 January
2023. 1. 1(일) : 신정
2023. 1. 2(월) ~ 6(금) : 복학 집중신청
2023. 1. 2(월) ~ 20(금) : 동계 계절학기
2023. 1. 21(토) ~ 23(월) (1. 24(화) 대체휴일) : 설날

구분 : 2 February
2023. 2. 2(목) ~ 3(금) : 장바구니 수강신청
2023. 2. 9(목) ~ 10(금) : 본 수강신청
2023. 2. 13(월) ~ 17(금) : 휴학 집중신청
2023. 2. 20(월) ~ 24(금) : 등록
2023. 2. 24(금) : 전기 학위수여식(제66회)
2023. 2. 28(화) : 신입생 입학식

구분 : 3 March
2023. 3. 1(수) : 삼일절
2023. 3. 2(목) : 개강
2023. 3. 2(목) ~ 8(수) : 수강신청 변경
2023. 3. 29(수) : 수업일수 4주선

구분 : 4 April

구분 : 5 May
2023. 5. 5(금) : 어린이날
2023. 5. 10(수) : 수업일수 2/3선

구분 : 6 June
2023. 5. 27(일) : 석가탄신일
2023. 6. 6(화) : 현충일
2023. 6. 8(목) ~ 14(수) : 보강기간
2023. 6. 15(목) ~ 22(목) : 기말시험
2023. 6. 16(금) : 개교기념일(제71회)

구분 : 7 July
2023. 7. 3(월) : 하계 계절학기 개강
2023. 7. 3(월) ~ 7(금) : 복학 집중신청
2023. 7. 21(금) : 하계 계절학기 종강

제 2 학기

8월:
- 3일(목) ~ 4일(금): 장바구니 수강신청
- 10일(목) ~ 11일(금): 본 수강신청
- 14일(월) ~ 18일(금): 휴학 집중신청
- 15일(화): 광복절
- 18일(금): 후기 학위수여식(제66회)
- 21일(월) ~ 25일(금): 등록기간
- 28일(월): 개강

9월:
- 28일(월) ~ 1일(금): 수강신청변경
- 24일(일): 수업일수 4주선
- 2


교양 이수학점1) 2023학번
대영역 : 공통교양
세부영역 : -
학점 : 10~13학점

대영역 : 학문기초교양
세부영역 : -
학점 : 13~21학점

대영역 : 선택교양
세부영역 : 
1) 철학과 역사
2) 문학과 예술
3) 인간과 사회
4) 자연과 과학
5) 세계와 문화
학점 : 9 학점 이상 (최소 3개 영역 선택)
비고 : 12 학점(1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수

대영역 : 선택교양
세부영역 : 
6) 소양
7) 자기계발
학점 : 자유이수

계 : 39 학점 이상
1-1) 2023학번 학부(과) 별 교양이수학점
학부/과 : 항공우주 및 기계공학부
공통교양 : 11
학문기초교양 : 21
선택교양 : 12
계 : 44

학부/과 : 항공전자정보공학부
공통교양 : 11
학문기초교양 : 20
선택교양 : 12
계 : 43

학부/과 : 신소재공학과
공통교양 : 11
학문기초교양 : 17
선택교양 : 12
계 : 40

학부/과 : 소프트웨어학과
공통교양 : 13
학문기초교양 : 13
선택교양 : 12
추가이수 교양학점* : 1
계 : 39

학부/과 : 스마트드론공학과
공통교양 : 10
학문기초교양 : 19
선택교양 : 12
계 : 41

학부/과 : AI자율주행시스템공학과
공통교양 : 12
학문기초교양 : 16
선택교양 : 12
계 : 40

학부/과 : 항공교통물류학부
공통교양 : 11
학문기초교양 : 15
선택교양 : 12
추가이수 교양학점* : 1
계 : 39

학부/과 : 항공운항학과
공통교양 : 11
학문기초교양 : 15
선택교양 : 12
추가이수 교양학점* : 1
계 : 39

학부/과 : 경영학부
공통교양 : 11
학문기초교양 : 15
선택교양 : 12
추가이수 교양학점* : 1
계 : 39
* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨   ※ 년도별 교양교과과정표를 확인하여 소속 학부(과)별 필수이수 과목을 확인할 것(한국항공대학교 인문


사회봉사 과목이수안내
1. 사회봉사 이수방법

이론교육(4시간) + 봉사활동(34시간 이상)

2. 사회봉사 운영
이수구분 : 교양 선택
학점인정 : 1학점
비고 : “P”(Pass), “NP”(Non-Pass)

※ 수강신청 시 본인 최대수강신청 학점에 미포함
※ 재학 중 1회 1학점씩 총 2회 2학점까지 이수 가능
※ 성적산출시 취득학점에 포함, 평점평균 산정에 미포함
※ 연속 2회 “NP”일 경우 수강신청 불가

3. 사회봉사활동 인정기간

수강신청 직전학기 기말시험 시작일 ~ 수강신청 학기 기말시험 전까지(※ 헌혈은 기간제한 없음)

4. 사회봉사활동 기관 인정 범위

가. 시·도 및 시·군·구 자원봉사센터, 청소년활동진흥센터, 기타 사회복지관, 지역아동센터, 사회복지법인 사회복지봉사활동인증관리(VMS) 및
1365자원봉사포털에 등록되어 ‘사회복지 봉사활동 실적 인증서(VMS)’ 및 ‘자원봉사활동 확인서(1365)’가 출력되는 기관

VMS 출력 시 필히 ‘봉사자 실적 내역정보’ 출력하여 제출

니. 비영리민간단체(봉사활동확인서 제출 시 ‘비영리민간단체등록증’ 사본 첨부)

다. 해외봉사 : 인증된 국제 NGO단체

NGO(Non-Governmental Organization 비정부기구)는 지역-국가-국제적으로 조직된 자발적인 비영리 시민단체

라. 자원봉사센터 및 시민단체와 연계하여 하는 농촌봉사활동

마. 총학생회 주관 자원봉사

1학기 : 항공산타, 신입생새로배움터, 은익제 등
2학기 : 항공제, 하계방학 중 실시 한 사회봉사활동 등

바. 기타 부서 주관 대외행사 및 교내 자원봉사

입학관리팀 ‘모교입시 설명회’, 국제교류부 ‘Global-Buddy’, 항공우주박물관 ‘안내봉사’ 등

사. 헌혈활동 : 헌혈증 제출

1회 기준 4시간 인정
1학기 최대 2회(8시간)까지만 인정
추후 본교 재학생 중 필요한 학생 및 저소득층, 백혈병 소아암 협회, 관련 사회복지기관 등에게'항공대 재학생' 이름으로 기증

5. 사회봉사활동 미인정 범위

가. 공공기

...

%%%%%%%%%%%%%%%%%%%
-------jj-------
58
['<html lang="ko">', '<head>', '</head>', '<body>', '<ul class="sub_panel">', '<li class="article_box_nrm" id="circles_01">', '<ul class="circles_list">', '<li>', '<div class="img_box">', '<span class="img"><img src="/cmm_typeA/images/sub/abs.png"/></span>', '<a href="http://www.kaupress.com" target="_blank">홈페이지 바로가기</a>', '</div>', '<h4><strong>한나래 방송국(ABS)</strong> (창립년도 : 1966 / 위치 : 학생회관 203호)</h4>', '<dl>', '<dt>설립목적</dt>', '<dd>한나래 방송국은 교내 유일의 방송 언론기관으로, 항공대 학우들에게 교내외 소식, 시사 정보는 물론 라디오 방송을 통해 다양한 주제의 정보를 목소리로 직접 전달합니다.</dd>', '</dl>', '<dl>', '<dt>활동내용</dt>', '<dd>교내 라디오 방송</dd>', '<dd>연 1회 한나래 방송제</dd>', '<dd>정보 전달 방송 관제탑 (youtube)</dd>', '</dl>', '</li>', '<li>', '<div class="img_box">', '<span class="img"><img src="/cmm_typeA/images/sub/noImg.jpg"/></span>', '<a href="http://kaupress.com" target="_blank">홈페이지 바로가기</a>', '</div>', '<h4><strong>항공대신문사</strong> (창립년도 : 1966 / 위치 : 학생회관 201호)</h4>', '<dl>', '<dt>설립목적</dt>', '<dd>항공대신문사는 196

%%%%%%%%%%%%%%%%%%%
-------jj-------
65
['<html lang="ko">', '<head>', '</head>', '<body>', '<dl class="motto_box">', '<dt class="sky"><span>건학정신</span></dt>', '<dd>개척자적 정신을 배양하고, 지성인으로서의 교양과 첨단의 과학기술로 무장하여, 새로운 시대의 역사적 소임을 다할 수 있는 능력있는 선도자를 배출</dd>', '</dl>', '<div class="space_sep">', '<img src="/cmm_typeA/images/sub/sep_arrD.png"/>', '</div>', '<dl class="motto_box">', '<dt class="blue"><span>교육이념<br/>(교훈)</span></dt>', '<dd>인격도야(人格陶冶) : 마음과 몸매를 맑게</dd>', '<dd>진리탐구(眞理探究) : 학문과 창의를 넓게</dd>', '<dd>인화협동(人和協同) : 화협과 단결을 굳게</dd>', '</dl>', '<div class="space_sep">', '<img src="/cmm_typeA/images/sub/sep_arrD.png"/>', '</div>', '<dl class="motto_box">', '<dt class="sky"><span>교육목적</span></dt>', '<dd>항공우주과학을 중심으로 국가사회와 인류발전에 필요한 다양한 학문분야에 관한 이론과 응용방법을 교수 및 연구지도자로서의 인격과 실력을 갖춘 유능한 인재양성</dd>', '</dl>', '<div class="space_sep">', '<img src="/cmm_typeA/images/sub/sep_arrD.png"/>', '</div>', '<dl class="motto_box">', '<dt class="blue"><span>교육목표</span></dt>', '<dd>항공우주과학을 중심으로 한 관련분야의 전문인 육성\xa0

%%%%%%%%%%%%%%%%%%%
-------jj-------
2014
['<html lang="ko">', '<head>', '</head>', '<body>', '<span id="husky_bookmark_start_1658909679987"></span><span id="husky_bookmark_end_1658909679987"></span><div class="organization_chart">', '<span class="org_top"><img src="/cmm_typeA/images/sub/org_top.png"/></span>', '<ul class="org_direct">', '<li><a href="http://college.kau.ac.kr/web/index.do?siteFlag=senator" target="_blank">대학평의원회</a></li>', '<li>비서실</li>', '</ul>', '<ul class="org_dpt">', '<li>', '<a class="prc" href="#none">대학본부</a>', '<ul class="org_dpt2">', '<li><a href="#" onclick="showInfo(\'#org1\'); return false;">교무처</a></li>', '<li><a href="#" onclick="showInfo(\'#org2\'); return false;">학생처</a></li>', '<li><a href="#" onclick="showInfo(\'#org3\'); return false;">기획처</a></li>', '<li><a href="#" onclick="showInfo(\'#org4\'); return false;">연구협력처</a></li>', '<li><a href="#" onclick="showInfo(\'#org5\'); return false;">입학처</a></li>', '<li><a href="#" onclick="showInfo(\'#org6\'); 


대학평의원회
비서실

대학본부

교무처
학생처
기획처
연구협력처
입학처
사무처

기타 기관

대학일자리센터
현장실습지원센터
학군단

대학

공과대학
AI융합대학
항공·경영대학
자유전공학부
인문자연학부

미래교육혁신원

교수학습센터
교육성과관리센터
원격교육지원센터

대학원

대학원
항공·경영대학원

산학협력단

중소기업 산학협력센터

차세대방송미디어 기술연구센터
디스플레이 기술연구센터

창업지원센터
공동기기센터

부속기관

학술정보관
전산정보원
인권센터
신문사
방송국
비행교육원
관제소
항공교통관제교육원
항공기술교육원
한국항공안전교육원
항공우주박물관
국제교류원
평생교육원
생활관
사회봉사단

연구소

항공전자연구소
항공우주산업 기술연구소
KAU로봇연구센터
신소재연구센터
기계항공산업신뢰성 기술연구센터
표면기술응용 연구센터
개도국기술경영 지원센터
교통물류연구소
항공산업정책연구소
항공안전관리연구소
유비쿼터스 기술응용 연구센터
항공체계시험 인증연구센터
첨단무인기 연구센터
국방광역감시 특화연구센터
미래항공모빌리티 기술센터

IT연구소
미래융합기술개발 연구센터

문의 : 기획홍보팀/김태웅
전화번호 : 02-300-0044
E-mail : taewungk@kau.ac.kr

교무처

개요

교무처 교무팀의 담당업무는 크게 학사 업무와 교원인사업무로 나눌 수 있다. 우선 학사 업무의 경우 교육과정, 수업, 평가 등 대학 학사관리 전반을 관장하여, 유연한 학사운영을 통한 인재 육성을 그 목표로 한다. 교원인사업무의 경우 전임 및 비전임교원의 임용, 인사관리, 평가 전반을 다루며, 교원의 교육 및 연구역량 강화를 통한 대학의 질적 발전을 목표로 한다.

담당분야
- 전임/비전임 교원의 임용, 평가 및 인사 전반에 관한 사항
- 학부 교육과정, 수업, 평가 등 학사 관리에 관한 사항
- 휴ㆍ복학, 졸업 등 학적관리에 관한 사항

담당업무/연락처

교무처

성명 : 배재성
직위/직급 : 교무처장
전화번호 : 02-300-0300
담당업무 : 교무처 총괄

성명 : 윤미경
직위/직급 :


학교법인
학칙
일반행정
교무 및 연구행정
학생 및 장학행정
부속 및 부설기관
위원회
기타

대학규정 : 정관
PDF 다운로드 : [링크]

대학규정 : 교원징계위원회 운영규정
PDF 다운로드 : [링크]

대학규정 : 일반직원징계위원회 운영규정
PDF 다운로드 : [링크]

대학규정에는 다음과 같은 내용이 포함되어 있습니다:

- 제 2편 학칙
    - 제 1장 대학교
        - 학칙
        - 학칙 시행세칙
    - 제 2장 대학원
        - 대학원 학칙
        - 일반 대학원 학칙 시행세칙
        - 항공 · 경영대학원 학칙 시행세칙

PDF 다운로드 링크는 다음과 같습니다:
- [제 1장 대학교 학칙](/doc/aboutkau/rules/2-1-01.pdf)
- [제 1장 대학교 학칙 시행세칙](/doc/aboutkau/rules/2-1-02.pdf)
- [제 2장 대학원 대학원 학칙](/doc/aboutkau/rules/2-2-01.pdf)
- [제 2장 대학원 일반 대학원 학칙 시행세칙](/doc/aboutkau/rules/2-2-02.pdf)
- [제 2장 대학원 항공 · 경영대학원 학칙 시행세칙](/doc/aboutkau/rules/2-2-03.pdf)

대학규정에는 다양한 규정들이 포함되어 있습니다. 몇 가지 예시를 들면, 제 3편 일반행정에는 조직 및 인사에 관한 규정들이 있습니다. 그 중에는 직제규정, 사무분장 규정, 위임전결 규정, 직원인사 규정 등이 있습니다. 또한, 제 4장 재무행정에는 교직원 보수 규정, 직원 재정 보증에 관한 내규, 강의료 지급 규정, 여비 규정 등이 포함되어 있습니다. 이 외에도 다양한 규정들이 있으며, 자세한 내용은 해당 규정의 PDF 파일을 다운로드하여 확인할 수 있습니다.

대학규정 - 제 4편 교무 및 연구행정 - 교원인사규정
대학규정 - 제 4편 교무 및 연구행정 - 교원인사규정 시행세칙
대학규정 - 제 4편 교무 및 연구행정 - 조종실기교수 인사 규정
대학규정 - 

In [ ]:
import re
import csv
import pandas as pd


def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
#     sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
#     sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences


page_indexList = []
pageList = []
data = pd.read_csv('test3.csv', header=None, encoding="UTF-8")
for i in range(len(data)):
    page_indexList.append(data.iloc[i][0])
    pageList.append(data.iloc[i][1])

temp_str = ""
for i in range(len(page_indexList)):
    if page_indexList[i] == 41:
        temp_page = split_sentences(pageList[i])
        for j in range(len(temp_page)):
            if "// 팝업 띄우기" in temp_page[j]:
                temp_str = temp_str.strip()
                break
            else:
                temp_str = temp_str + temp_page[j] + "\n"
        temp_str = temp_str.strip()
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], temp_str])
        f.close()    
        temp_str = ""
    else:
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], pageList[i]])
        f.close()    
        
    

In [ ]:
import os
import openai

openai.api_key ="API_KEY"
table_g = """                            <table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>-</td><td colspan="2">10~13학점</td></tr><tr><td>학문기초교양</td><td>-</td><td colspan="2">13~21학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양<br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div></h5><h5>1-1) 2023학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td>21</td><td>12</td><td>-</td><td>44</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td>20</td><td>12</td><td>-</td><td>43</td></tr><tr><td>신소재공학과</td><td>11</td><td>17</td><td>12</td><td>-</td><td>40</td></tr><tr><td>소프트웨어학과</td><td>13</td><td>13</td><td>12</td><td>1</td><td>39</td></tr><tr><td>스마트드론공학과</td><td>10</td><td>19</td><td>12</td><td>-</td><td>41</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>16</td><td>12</td><td>-</td><td>40</td></tr><tr><td>항공교통물류학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>항공운항학과</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>경영학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr></tbody></table></div></h5>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<strong>&nbsp;&nbsp;&nbsp;<br><span style="color: rgb(0, 0, 0);"><span style="color: rgb(58, 50, 195);">※ 년도별 교양교과과정표를 확인하여 소속 학부(과)별 필수이수 과목을 확인할 것</span>(</span></strong><a href="http://college.kau.ac.kr/web/index.do?siteFlag=ct_www"><span style="color: rgb(0, 0, 0);">한국항공대학교 인문자연학부 (kau.ac.kr)</span></a><span style="color: rgb(0, 0, 0);">)</span><strong><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </strong><h5><br>2) 2022학번<br><br> <div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">11~13학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </h5><h5>2-1) 2022학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td> 20 </td><td> 12 </td><td>- </td><td> 43</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td> 18</td><td> 12 </td><td> - </td><td>41</td></tr><tr><td>신소재공학과</td><td>11</td><td> 15</td><td> 12 </td><td>1</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>13</td><td> 12</td><td> 12 </td><td>2</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>12</td><td> 18</td><td> 12 </td><td>-</td><td>42</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>15</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>항공교통물류학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>항공운항학과</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>경영학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br><br>3) 2021학번<br><br> </p><div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">9~11학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </div></h5><h5>3-1) 2021학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>9 </td><td> 20 </td><td> 12 </td><td>- </td><td> 41</td></tr><tr><td>항공전자정보공학부</td><td>9 </td><td> 18</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>신소재공학과</td><td>9</td><td> 15</td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>11</td><td>12</td><td> 12 </td><td>4</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>10</td><td>15</td><td> 12 </td><td>2</td><td> 39</td></tr><tr><td>항공교통물류학부</td><td>9</td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>항공운항학과</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>경영학부</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br>4) 2020 학번</p></div></h5><strong>
                            </strong><strong>
                                </strong><strong>
                                    </strong><strong>
                                    </strong><strong>
                                    </strong><table class="table_nrm center"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th>
                                </tr>
                                <tr>
                                    <td>공통교양</td>
                                    <td>-</td>
                                    <td colspan="2">7 학점</td>
                                </tr>
                                <tr>
                                    <td>학문기초교양</td>
                                    <td>-</td>
                                    <td colspan="2">12 ~ 20학점, 각 학부(과)별 상이</td>
                                </tr>
                                <tr>
                                    <td rowspan="2">선택교양</td>
                                    <td>
                                        1) 철학과 역사<br>
                                        2) 문학과 예술<br>
                                        3) 인간과 사회<br>
                                        4) 자연과 과학<br>
                                        5) 세계와 문화
                                    </td>
                                    <td>9 학점 이상<br>(최소 3개 영역 선택)</td>
                                    <td rowspan="2">12 학점<br>※ 1) ~ 5) 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수</td>
                                </tr>
                                <tr>
                                    <td>
                                        6) 소양<br>
                                        5) 자기계발
                                    </td>
                                    <td>자유이수</td>
                                </tr>
                                <tr>
                                    <th colspan="2">계</th>
                                    <th colspan="2">39 학점 이상</th>
                                </tr>
                            </tbody></table>"""

sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>
---------
답 : 
---------
편의시설 : 식당
위치 : 1층 학생식당 내
전화번호 : -

편의시설 : 편의점(GS)
위치 : 1층 학생식당 내
전화번호 : 02-3158-0336

편의시설 : 우리은행
위치 : 1층(운영시간) 09:30~15:30
전화번호 : 02-3158-3656

편의시설 : 블루포트(take out coffee)
위치 : 1층 중앙로비
전화번호 : -
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
if answer.endswith('"'):
    answer = answer[:-1]
    answer = answer
print(answer)

In [ ]:
ass1 = """
함수 이름: [table_to_text]
입력: ["<table></table>"]
규칙: [당신이 정리하는 역할을 하길 바랍니다. 나는 <table> 태그를 포함한 입력 양식을 제공할 것이며 당신은 <table>을 감지하고 텍스트로 정리할것입니다.]
table_to_text(<table class="table_nrm center">
                                    <tbody><tr>
                                        <th rowspan="2">증명서 종류</th>
                                        <th rowspan="2">발급방법</th>
                                        <th colspan="2">수수료</th>
                                        <th rowspan="2">발급 시 관련부서</th>
                                    </tr>
                                    <tr>
                                        <th>국문</th>
                                        <th>영문</th>
                                    </tr>
                                    <tr>
                                        </tr><tr>
                                            <td>성적/재학/휴학/재적증명서<br>제적/졸업예정/졸업/수료증명서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>주민센터 내 FAX민원, <br>우체국 우편발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>학적부, 사실증명원</td>
                                            <td>방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>장학수혜확인서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>* 대학원은 방문 발급만 해당</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>교육비납입증명서</td>
                                            <td>종합정보시스템에서 출력가능</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>재무팀</td>
                                        </tr>
                                        <tr>
                                            <td>교외장학지원 관련 추천서</td>
                                            <td>방문발급</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>실무경력확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공교통관제교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행교육원 이수증명서</td><td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>공학교육인증 참여확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>공학교육혁신센터</td>
                                        </tr>
                                        <tr>
                                            <td>항공정비사 정비실습확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공우주 및 기계공학부</td>
                                        </tr>
                                        <tr>
                                            <td>운항관리경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원<br></td>
                                        </tr>
                                </tbody></table>)
"""

In [ ]:
import os
import openai

openai.api_key ="API_KEY"


# 테이블 태그 임의로 지정하기
table_g = """<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>"""


# (텍스트 + 테이블 태그)를 입력으로 주었을 때 문제가 발생하는지 파악
full = f"""               

사회봉사 과목이수안내
1. 사회봉사 이수방법
이론교육(4시간) + 봉사활동(34시간 이상)

{table_g}

※ 수강신청 시 본인 최대수강신청 학점에 미포함
※ 재학 중 1회 1학점씩 총 2회 2학점까지 이수 가능
※ 성적산출시 취득학점에 포함, 평점평균 산정에 미포함
※ 연속 2회 “NP”일 경우 수강신청 불가

3. 사회봉사활동 인정기간
수강신청 직전학기 기말시험 시작일 ~ 수강신청 학기 기말시험 전까지(※ 헌혈은 기간제한 없음)

4. 사회봉사활동 기관 인정 범위
가. 시·도 및 시·군·구 자원봉사센터, 청소년활동진흥센터, 기타 사회복지관, 지역아동센터, 사회복지법인 사회복지봉사활동인증관리(VMS) 및
       1365자원봉사포털에 등록되어 ‘사회복지 봉사활동 실적 인증서(VMS)’ 및 ‘자원봉사활동 확인서(1365)’가 출력되는 기관
VMS 출력 시 필히 ‘봉사자 실적 내역정보’ 출력하여 제출
니. 비영리민간단체(봉사활동확인서 제출 시 ‘비영리민간단체등록증’ 사본 첨부)
다. 해외봉사 : 인증된 국제 NGO단체
NGO(Non-Governmental Organization 비정부기구)는 지역-국가-국제적으로 조직된 자발적인 비영리 시민단체
라. 자원봉사센터 및 시민단체와 연계하여 하는 농촌봉사활동
마. 총학생회 주관 자원봉사
1학기 : 항공산타, 신입생새로배움터, 은익제 등
2학기 : 항공제, 하계방학 중 실시 한 사회봉사활동 등
바. 기타 부서 주관 대외행사 및 교내 자원봉사
입학관리팀 ‘모교입시 설명회’, 국제교류부 ‘Global-Buddy’, 항공우주박물관 ‘안내봉사’ 등
사. 헌혈활동 : 헌혈증 제출
1회 기준 4시간 인정
1학기 최대 2회(8시간)까지만 인정
추후 본교 재학생 중 필요한 학생 및 저소득층, 백혈병 소아암 협회, 관련 사회복지기관 등에게'항공대 재학생' 이름으로 기증

5. 사회봉사활동 미인정 범위
가. 공공기관, 행정기관에서의 봉사활동
공공기관 : 주민센터(구. 동사무소), 시.도.군.구청, 경찰서, 세무서, 소방서, 우체국 등
정부행정기관 : 국세청, 법원, 감사원, 국회 등
나. 도서관(단, 시각장애인을 위한 점자도서관은 인정), 어린이집, 경로당, 종교관련 봉사활동
다. 시·군·구 개최, 지역 개최 봉사활동(단, 교육과학기술부에서 공문화 되어 온 봉사활동은 인정)
라. 제출기간까지 봉사활동확인서가 미발급되거나 기간 및 시간이 정확히 기재되지 않은 봉사활동
   (예 : 중앙일보 '공부의 신')

6. 제출서류
가. “사회봉사”과목 학점인정 서류
수강신청 전원 제출, 미제출시 학점인정 안됨
본교 총학생회 주관 자봉단 활동 시에도 제출
나. 사회봉사활동확인서(봉사활동을 실시한 기관에 양식이 없을 경우 사용)

7. 유의사항
가. 필히 ‘이론교육’ 4시간을 이수해야 봉사활동확인서 제출 시 학점 인정
나. 봉사활동 확인서는 원본을 제출하여야 하며, 제출한 서류는 일체 반환하지 않음
다. 봉사활동 확인서 수정 금지 (수정 시 담당자 날인이 없을 경우 무효 처리)
    * 문의사항 : 02-300-0014
"""


# 테이블과 관련된 질문하기
question ="""혹시 편의시설에 대해 알려줄래?"""



sys = f"""{full}

{question}"""

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
# answer = answer.replace('\"',"")
print(answer)

In [ ]:
import pandas as pd
import openai
import csv
import numpy as np
from time import sleep
import json
import ast
import re
import random
from itertools import *
import sys
delay = 2

openai.api_key ="API_KEY"

def get_embedding(text, model="text-embedding-ada-002"):
    count = 0 # 예외 발생 횟수를 저장할 변수
    max_count = 5 # 예외가 발생한 최대 횟수
    while True:
        try :
            temp = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
            count=0
            return temp
        except Exception as e:
            count += 1
            print(count)
            if count >= max_count:
                print("예외가 연속으로 %d번 발생했습니다. 프로그램을 종료합니다." % max_count)
                break
            else:
                print("예외가 발생했습니다. (%s)" % e)
def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

                
Sec_index = []
SecList = []
    
csv_name = "test3_filter"

data = pd.read_csv(f'{csv_name}.csv', header=None, encoding="UTF-8")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    
###########################################################################################
temp_str = ""
for i in range(len(Sec_index)):
    print(f"------------  {i}  -------------")
    temp_manual = SecList[i]
    splitList = split_sentences(temp_manual)
    for j in range(len(splitList)):
        pattern = re.compile(r'\d+\.')
        matches = re.findall(pattern, temp_sentence)
        if matches:
            temp_str = temp_str + splitList[j]
        print(splitList[j])
        print("######################")
        